In [15]:
#5.4 단순한 계층 구현하기
#5.4.1 곱셈 계층

class MulLayer:
    def __init__(self): #__init__() 인스턴스 변수인 x와 y를 초기화. __init__함수를 사용하는 이유는?? 생략해도 차이 없는 거 아닌가??
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y

        return out
    
    def backward(self, dout):
        dx = dout * self.y  #x와 y를 바꾼다.
        dy = dout * self.x  #상류에서 넘어온 미분(dout)에 순전파 때의 값을 '서로 바꿔' 곱한 후 하류로 흘림

        return dx, dy
    
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()    #MulLayer(곱셈계층) class를 활용하는 객체 생성 
mul_tax_layer = MulLayer()  #MulLayer(곱셈계층) class를 활용하는 객체 생성 

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num) #객체.메소드(매개변수)를 활용해 apple_price 정의
price = mul_tax_layer.forward(apple_price, tax) #객체.메소드(매개변수)를 활용해 price 정의

print(price)

#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice) #순전파 과정을 거꾸로. 속성(?)명에는 d를 붙이고, forward를 backward로 변경 
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [12]:
#5.4.2 덧셈 계층
# Q. 곱셈계층과 덧셈계층이 필요한 경우는 어떻게 다른가? 왜 2가지가 필요한가?(ax + b) 역전파에서 *1인거지?

class AddLayer: #덧셈계층 클래스 생성
    def __init__(self): 
        pass    #pass 할 건데 그냥 def __init__(self) 함수 정의를 안 하면 안되나??

    def forward(self, x, y):    #순전파 메소드 정의
        out = x + y
        return out
    
    def backward(self, dout):   #역전파 메소드 정의
        dx = dout * 1
        dy = dout * 1
        return dx, dy

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()    #사과와 오렌지 가격을 구하는 층은 곱셈계층으로 정의
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer() #사과와 오렌지 가격의 합을 구하는 계층은 덧셈계층으로 정의
mul_tax_layer = MulLayer()  #세금 곱해주는 층은 곱셈계층으로 정의

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num) 
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(price)

#역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)   #순전파 역순으로. 변수에 d붙이고, forward대신 backwrad 메소드 활용
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)


print(dapple_num, dapple, dorange, dorange_num, dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [19]:
#5.5 활성화 함수 계층 구현하기
#5.5.1 ReLU 계층

class Relu:
    def __init__(self):
        self.mask = None

        def forward(slef, x):
            self.mask = (x <= 0)    #문법이 이해가 잘 안됨. x<=0인 경우 True, x>0인 경우 False 출력한다는 의미인 듯
            out = x.copy()
            out[self.mask] = 0

            return out
        
        def backward(self, dout):
            dout[self.mask] = 0
            dx = dout

            return dx
        
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [21]:
#5.5.2 Sigmoid 계층

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

In [32]:
#5.6 Affine/Softmax 계층 구현하기
#5.6.1 Affine 계층
#신경망의 행렬의 곱, 넘파이에서는 np.dot()을 사용

X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

X.shape #(2, )
W.shape #(2, 3)
B.shape #(3, )

Y = np.dot(X, W) + B    #행렬의 곱에서는 대응하는 차원의 수를 일치시킴. x.shape의 벡터(?) 2와 w.shape의 2가 동일. w.shape의 벡터(?) 3이 결과값과 동일.Y 벡터도 3.
print(Y)


[1.20573895 0.60933957 1.52247565]


In [40]:
#5.6.2 배치용 Affine 계층
#5.6.1의 Affine 계층은 입력 데이터로 X하나만을 고려. 이번에는 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층 계산식

X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print('-'*50)
print(X_dot_W + B) # 순전파의 편향 덧셈은 각각의 데이터에 더해짐
print('-'*50)


dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)
print('-'*50)

dB = np.sum(dY, axis = 0)
print(dB)
print('-'*50)


# Affine 계층 정리

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = B
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + B
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)    #T는 어디서 나온거지??
        self.db = np.sum(dout, axis = 0)    

        return dx

[[ 0  0  0]
 [10 10 10]]
--------------------------------------------------
[[ 1  2  3]
 [11 12 13]]
--------------------------------------------------
[[1 2 3]
 [4 5 6]]
--------------------------------------------------
[5 7 9]


In [42]:
#5.6.3 Softmax-with-Loss 계층
#Softmax 함수는 입력 값을 저규화하여 출력함. 출력값을 모두 더하면 1

#신경망에서 수행하는 작업은 학습과 추론. 신경망 학습에는 softmax 계층 필요
#하지만 추론에는 softmax함수를 사용하지 않음. 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 softmax 계층은 필요 없음. 앞선 계층에서 이미 최대값이 나옴

#소프트맥스 함수의 손실함수로 교차엔트로피오차를 사용하면 역전파가(y1-t1, y2-t2, y3-t3)로 떨어짐. y는 소프트맥스 함수결과, t는 정답
#신경망의 역전파에서 이 차이인 오차가 앞 계층에 전해지는 것임 

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None    #손실
        self.y = None       #softmax의 출력
        self.t = None       #정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t)/batch_size   #역전파 때는 전파하는 값을 배치의 수(batch_size)로 나눠서 데이터 1개당 오차를 앞 계층으로 전파 

In [18]:
#5.7.2 오차역전파법을 적용한 신경망 구현하기
#common 모듈 설치 오류로 마지막 장은 포기...ㅠ.ㅠ


# !pip install common

# !python -m venv myenv 
# !source myenv/bin/activate
# !pip install common

# !pip show common #Location: C:\Users\Sueun Kwon\AppData\Local\Programs\Python\Python311\Lib\site-packages


import sys
import os
sys.path.append(os.pardir)
sys.path.append('C:/Users/Sueun Kwon/AppData/Local/Programs/Python/Python311/Lib/site-packages')
# sys.path.insert(0,'C:/Users/Sueun Kwon/AppData/Local/Programs/Python/Python311/Lib/site-packages')
import numpy as np
# from common.layers import *
# from common.gradient import numerical_gradient
# from collections import OrderedDict

# class TwoLayerNet:
 
#     def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
#         # 가중치 초기화
#         self.params = {}
#         self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
#         self.params['b1'] = np.zeros(hidden_size)
#         self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
#         self.params['b2'] = np.zeros(output_size)
 
#         # 계층 생성
#         # OrderedDict = 순서가 있는 딕셔너리, 순서 기억
#         # 순전파 때는 계층을 추가한 순서대로 / 역전파 때는 계층 반대 순서로 호출
#         self.layers = OrderedDict()
#         self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
#         self.layers['Relu1'] = Relu()
#         self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
 
#         self.lastLayer = SoftmaxWithLoss()
        
#     def predict(self, x):
#         for layer in self.layers.values():
#             x = layer.forward(x)
        
#         return x
        
#     # x: 입력데이터, t : 정답레이블
#     def loss(self, x, t):
#         y = self.predict(x)
#         return self.lastLayer.forward(y, t)
    
#     def accuracy(self, x, t):
#         y = self.predict(x)
#         y = np.argmax(y, axis=1)
#         if t.ndim != 1 : t = np.argmax(t, axis=1)
        
#         accuracy = np.sum(y == t) / float(x.shape[0])
#         return accuracy
        
#     # x: 입력데이터, t : 정답레이블
#     def numerical_gradient(self, x, t):
#         loss_W = lambda W: self.loss(x, t)
        
#         grads = {}
#         grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
#         grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
#         grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
#         grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
#         return grads
        
#     def gradient(self, x, t):
#         # forward, 순전파
#         self.loss(x, t)
 
#         # backward, 역전파
#         dout = 1
#         dout = self.lastLayer.backward(dout)
        
#         layers = list(self.layers.values())
#         layers.reverse()
#         for layer in layers:
#             dout = layer.backward(dout)
 
#         # 결과 저장
#         grads = {}
#         grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
#         grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
 
#         return grads

ModuleNotFoundError: No module named 'common.gradient'